## scipy.optimize.linprogを利用した線形計画法

  scipy.optimize.linprogという線形計画法向けのソルバーを利用していきます。  
  scipy.optimizeは非線形も解けますが、微分の関係でwarningが出るので、素直にlinprogを使います。

例題1.1 Maximize $z = 2x_1 + 3x_2$   
Subject to :  
$2x_1 + x_2 \leq 100$  
$3x_1 + 6x_2 \leq 240$  
$x_1 \geq 0$  
$x_2 \geq 0$  

例題で学ぶOR入門 P29 例題3.1  
最適値は、$(x_1, x_2) = (40, 20)$のとき、最大値$140$

制約行列  

制約条件(線形制約)  
$  
\left(
\begin{array}{r}
-\infty \\
-\infty \\
-\infty \\
-\infty \\
\end{array}
\right)
\leq
\left(
\begin{array}{rr}
2 & 1 \\
3 & 6 \\
-1 & 0 \\
0 & -1 \\
\end{array}
\right)
\left(
\begin{array}{r}
x_1 \\
x_2 \\
\end{array}
\right)
\leq 
\left(
\begin{array}{r}
100 \\
240 \\
0 \\
0 \\
\end{array}
\right)
$

このときのポイントは  
$Ax \leq b$  
$Ax = b$  
だけで表現することです。また目的関数は最小化されるので、最大値問題の場合はマイナスをかけて最小値問題に置き換えて行います。

In [1]:
import numpy as np
from scipy.optimize import linprog

In [2]:
c = np.array([-2, -3])
A = np.array([[2, 1], [3, 6], [-1, 0], [0, -1]])
b = np.array([100, 240, 0, 0])
res = linprog(c, A_ub=A, b_ub=b, options={"disp": True})
print(res['x'])

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -5.0                
0.03050011289233    0.03050011289233    0.03050011289239    0.9697104672667  0.03050011289233    -71.93808045697     
0.002792338711531   0.002792338711534   0.00279233871154    0.9467233884814  0.002792338711539   -134.6632433178     
2.401457127308e-06  2.401457127262e-06  2.401457127377e-06  0.9991963257798  2.401457127215e-06  -139.9947060547     
1.200722935482e-10  1.200728986842e-10  1.200728405593e-10  0.999949999982   1.200740804108e-10  -139.9999997353     
Optimization terminated successfully.
         Current function value: -140.000000 
         Iterations: 4
[39.99999994 19.99999995]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


例題1.2 等式制約を含む場合  
https://coin-or.github.io/pulp/CaseStudies/a_blending_problem.html  
A Blending Problem  

Minimize $0.013x_1 + 0.008x_2$  
Subject to :  
$x_1 + x_2 = 100$  
$0.1x_1 + 0.2x_2 \geq 8.0$  
$0.08x_1 + 0.1x_2 \geq 6.0$  
$0.001x_1 + 0.005x_2 \leq 2.0$  
$0.002x_1 + 0.005x_2 \leq 0.4$  

行列表現   
$  
\left(
\begin{array}{r}
-\infty \\
-\infty \\
-\infty \\
-\infty \\
-\infty \\
-\infty \\
\end{array}
\right)
\leq
\left(
\begin{array}{rr}
-0.1 & -0.2 \\
-0.08 & -0.1 \\
0.001 & 0.005 \\
0.002 & 0.005 \\
-1 & 0 \\
- & -1 \\
\end{array}
\right)
\left(
\begin{array}{r}
x_1 \\
x_2 \\
\end{array}
\right)
\leq 
\left(
\begin{array}{r}
-8.0 \\
-6.0 \\
2.0 \\
0.4 \\
0.0 \\
0.0 \\
\end{array}
\right)
$

In [3]:
c = np.array([0.013, 0.008]) #目的関数
#不等式制約
A = np.array([[-0.1, -0.2], [-0.08, -0.1], [0.001, 0.005], [0.002, 0.005],[-1, 0],[0, -1]])
b = np.array([-8.0, -6.0, 2.0, 0.4, 0, 0])
#等式制約
Aeq = np.array([[1, 1]]) #１つの制約でも2次元で作成する必要がある
beq = np.array([100])
res = linprog(c, A_ub=A, b_ub=b, A_eq=Aeq, b_eq=beq, options={"disp": True})
print(res['x'])

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 0.021               
0.1219617629984     0.1219617629984     0.1219617629988     0.8797350120124  0.1219617629984     0.1726079403238     
0.01328247227275    0.01328247227281    0.01328247227285    0.9168230491442  0.01328247227281    0.7602685634634     
0.0009327336298774  0.000932733629882   0.0009327336298847  0.9625350734784  0.0009327336298821  1.129193151321      
0.0001279680203924  0.0001279680203931  0.0001279680203935  0.8645719452021  0.0001279680203931  1.101457792006      
1.226743411475e-05  1.226743414732e-05  1.226743414736e-05  0.9190765250751  1.22674341472e-05   0.9661419066737     
1.769104135703e-09  1.769104048378e-09  1.769104037004e-09  0.9999861159128  1.769104051326e-09  0.9666670763034     
8.848272881016e-14  8.845525709397e-14  8.845900396989e-1

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


例題1.3 boundsを使う例   
Minimize $f(x_1, x_2) = - x_1 + 4x_2$  
Subject to :  
$-3 x_1 + x_2 \leq 6$  
$x_1 + 2x_2 \leq 4$  
$-\infty \leq x_1 \leq \infty$  
$x_2 \geq -3$  
https://docs.scipy.org/doc/scipy-0.18.1/reference/generated/scipy.optimize.linprog.html  
最適値は $(x_1, x_2) = (10, -3)$のとき、最小値$-22$

制約条件の行列化  
$  
\left(
\begin{array}{r}
-\infty \\
-\infty \\
-\infty \\
-\infty \\
\end{array}
\right)
\leq
\left(
\begin{array}{rr}
-3 & 1 \\
1 & 2 \\
1 & 0 \\
0 & -1 \\
\end{array}
\right)
\left(
\begin{array}{r}
x_1 \\
x_2 \\
\end{array}
\right)
\leq 
\left(
\begin{array}{r}
6 \\
4 \\
\infty \\
3 \\
\end{array}
\right)
$

In [4]:
c = np.array([-1, 4])
A = np.array([[-3, 1], [1, 2]])
b = np.array([6, 4])
x0_bounds = (None, None)
x1_bounds = (-3, None)
res = linprog(c, A_ub=A, b_ub=b, bounds=(x0_bounds, x1_bounds),options={"disp": True})
print(res['x'])
#この場合はboundsで制限を入れないと正しい値が出てこない。制約の無い変数がある場合はboundsが必要になる？

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 -8.0                
0.09885158404625    0.09885158404625    0.09885158404625    0.903461537018   0.09885158404625    -6.284698425658     
0.05788429348353    0.05788429348355    0.05788429348355    0.4273037994111  0.05788429348355    -7.864724729573     
0.04539867008243    0.04539867008244    0.04539867008244    0.2387091287399  0.04539867008244    -12.78916804766     
0.00666151448168    0.006661514481681   0.006661514481682   0.8665142913493  0.006661514481682   -21.3520715063      
6.299626472385e-06  6.299626472585e-06  6.299626472583e-06  1.0              6.299626472588e-06  -21.99681708159     
3.150192408518e-10  3.150193669765e-10  3.150193217394e-10  0.9999499939647  3.150193297574e-10  -21.99999984082     
Optimization terminated successfully.
         Current fu

/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


坂和、西崎 : 数理計画法入門、森北出版 P14 例2.3.  
Minimize $f(x_1, x_2) = 4x_1 + 3x_2$  
Subject to:  
$x_1 + 3x_2 \geq 12$  
$x_1 + 2x_2 \geq 10$  
$2x_1 + x_2 \geq 15$  
$x_1 \geq 0, x_2 \geq 0$

行列表現   
$  
\left(
\begin{array}{r}
-\infty \\
-\infty \\
-\infty \\
-\infty \\
-\infty \\
\end{array}
\right)
\leq
\left(
\begin{array}{rr}
-1 & -3 \\
-1 & -2 \\
-2 & -1 \\
-1 & 0 \\
0 & -1 \\
\end{array}
\right)
\left(
\begin{array}{r}
x_1 \\
x_2 \\
\end{array}
\right)
\leq 
\left(
\begin{array}{r}
-12 \\
-10 \\
-15 \\
0 \\
0 \\
\end{array}
\right)
$

最適解は $(x_1, x_2) = (6.6, 1.8)$のとき最小値31.8


In [6]:
c = np.array([4, 3])
A = np.array([[-1, -3], [-1, -2], [-2, -1], [-1, 0], [0, -1]])
b = np.array([-12, -10, -15, 0, 0])
res = linprog(c, A_ub=A, b_ub=b, options={"disp": True})
print(res['x'])

Primal Feasibility  Dual Feasibility    Duality Gap         Step             Path Parameter      Objective          
1.0                 1.0                 1.0                 -                1.0                 7.0                 
0.154899053795      0.154899053795      0.154899053795      0.8512510615871  0.154899053795      23.61299642609      
0.01433267460306    0.01433267460311    0.01433267460311    0.9246834839962  0.01433267460315    31.21892194089      
0.001401050533693   0.001401050533705   0.001401050533704   0.9039255058703  0.001401050533694   31.72724577748      
3.879357477015e-06  3.879357460685e-06  3.879357460601e-06  1.0              3.879357500142e-06  31.79982783856      
1.939998367287e-10  1.940010301911e-10  1.940012595014e-10  0.9999499914527  1.939952975386e-10  31.79999999139      
Optimization terminated successfully.
         Current function value: 31.800000   
         Iterations: 5
[6.6 1.8]


/usr/local/lib/python3.7/dist-packages/numpy/core/fromnumeric.py:87: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
